In [22]:
#загрузка библиотек
import pymupdf
import fitz
import pandas as pd
import numpy as np

In [23]:
filename = 'Vals-Sac-31-I.pdf'
doc = pymupdf.open(filename)

In [24]:
#считывание файла эксель
data_to_change=pd.read_excel('Date time (1) (6).xlsx')

In [25]:
def redact_text_on_page(page, df, page_number):
    """
    Заменяет текст на указанной странице документа.

    :param page: Объект страницы документа
    :param df: DataFrame с данными для замены
    :param page_number: Номер страницы для обработки
    """
    # Фильтруем данные для текущей страницы
    df_page = df[df['Page'] == page_number]

    for i, raw_text in enumerate(df_page['Old Value'].values):
        new_text = str(df_page['New Value'].values[i])
        hits = page.search_for(str(raw_text))

        # Параметры для редактирования
        new_fontsize = 20  # Новый размер шрифта
        new_width = -0.1    # Новая ширина прямоугольника
        new_width_2 = -14   # Новая ширина прямоугольника
        new_height = -1.1   # Новая высота прямоугольника
        new_height_2 = -0.85   # Новая высота прямоугольника

        for rect in hits:
            x1, y1, x2, y2 = rect
            new_x1 = x1 + new_width_2
            new_x2 = x2 + new_width
            new_y2 = y2 - new_height_2*1.1
            new_y1 = y1 + new_height
            new_rect = fitz.Rect(new_x1,new_y1, new_x2, new_y2)
            
            page.apply_redactions(text=1)
            
            # Добавляем аннотацию для редактирования
            page.add_redact_annot(new_rect, new_text,
                                   fontname=page.get_fonts()[1][4],
                                   fontsize=new_fontsize,
                                   align=fitz.TEXT_ALIGN_RIGHT)
            

        # Применяем редактирование
        page.apply_redactions()


In [26]:
for page_number in [0,3]:
    page = doc[page_number]
    redact_text_on_page(page, data_to_change, page_number)


In [27]:
doc.save("document.pdf")  # Замените на нужное имя файла
doc.close()